# Ensemble Learning

## Initial Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

In [8]:
from sklearn.model_selection import train_test_split
from imblearn.ensemble import BalancedRandomForestClassifier
from imblearn.ensemble import EasyEnsembleClassifier

## Read the CSV and Perform Basic Data Cleaning

In [4]:
# Load the data


file_path = Path('~/desktop/Jupyter Workspace/Homework11//LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)

# Preview the data
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [7]:
label_encoder = LabelEncoder()

label_encoder.fit(df["home_ownership"])
df["home_ownership"] = label_encoder.transform(df["home_ownership"])

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,3,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,1,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,1,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,3,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,1,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


## Split the Data into Training and Testing

In [9]:
# Create our features
X = pd.get_dummies(df.drop(columns = "loan_status", axis = 1))

# Create our target
y = df["loan_status"]

In [10]:
X.describe()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,...,issue_d_Mar-2019,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
count,68817.000000,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.0,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,68817.0
mean,16677.594562,0.127718,480.652863,1.812779,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,...,0.177238,1.0,0.123879,0.876121,0.383161,0.616839,0.860340,0.139660,1.0,1.0
std,10277.348590,0.048130,288.062432,0.941313,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,...,0.381873,0.0,0.329446,0.329446,0.486161,0.486161,0.346637,0.346637,0.0,0.0
min,1000.000000,0.060000,30.890000,0.000000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,...,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.0
25%,9000.000000,0.088100,265.730000,1.000000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,...,0.000000,1.0,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.0,1.0
50%,15000.000000,0.118000,404.560000,1.000000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,...,0.000000,1.0,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.0,1.0
75%,24000.000000,0.155700,648.100000,3.000000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,...,0.000000,1.0,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.0,1.0
max,40000.000000,0.308400,1676.230000,3.000000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,...,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0


In [11]:
# Check the balance of our target values
y.value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [12]:
# Split the X and y into X_train, X_test, y_train, y_test
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1, 
                                                    stratify=y)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [13]:
# Create the StandardScaler instance
data_scaler = StandardScaler()

In [14]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = data_scaler.fit(X_train)

In [15]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [16]:
# Resample the training data with the BalancedRandomForestClassifier
brf_model = BalancedRandomForestClassifier(n_estimators=500, random_state=1)
brf_model = brf_model.fit(X_train_scaled, y_train)

In [17]:
# Calculated the balanced accuracy score
y_pred_rf = brf_model.predict(X_test)
balanced_accuracy_score(y_test, y_pred_rf)

0.6031128757387868

In [18]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred_rf)

array([[   20,    67],
       [  405, 16713]], dtype=int64)

In [19]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred_rf))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.05      0.23      0.98      0.08      0.47      0.21        87
   low_risk       1.00      0.98      0.23      0.99      0.47      0.24     17118

avg / total       0.99      0.97      0.23      0.98      0.47      0.24     17205



In [20]:
# List the features sorted in descending order by feature importance
importances = brf_model.feature_importances_

sorted(zip(brf_model.feature_importances_, X.columns), reverse=True)

[(0.07272525262201103, 'total_rec_prncp'),
 (0.06406132608841178, 'total_rec_int'),
 (0.06114476841090468, 'last_pymnt_amnt'),
 (0.05472682446109891, 'total_pymnt'),
 (0.05403466418427294, 'total_pymnt_inv'),
 (0.028981475119067658, 'int_rate'),
 (0.019080874459045095, 'issue_d_Jan-2019'),
 (0.017842747845291464, 'dti'),
 (0.016922558443138603, 'bc_util'),
 (0.016730597493687164, 'max_bal_bc'),
 (0.016435082864634323, 'annual_inc'),
 (0.016243353306690345, 'installment'),
 (0.015926112148083888, 'out_prncp_inv'),
 (0.015897595948970365, 'mths_since_recent_inq'),
 (0.015406620061817663, 'out_prncp'),
 (0.015331586878384295, 'il_util'),
 (0.015179564027240538, 'avg_cur_bal'),
 (0.015164026530238238, 'mo_sin_old_rev_tl_op'),
 (0.014129488241587452, 'mo_sin_old_il_acct'),
 (0.014005306370748474, 'revol_bal'),
 (0.01371871328108811, 'total_rev_hi_lim'),
 (0.01357863192186352, 'tot_cur_bal'),
 (0.013557255632210531, 'bc_open_to_buy'),
 (0.013448092594054934, 'total_bc_limit'),
 (0.0133713021

### Easy Ensemble Classifier

In [21]:
# Train the Classifier
eec_model = EasyEnsembleClassifier(n_estimators=100, random_state=1)
eec_model = eec_model.fit(X_train, y_train)
eec_model

EasyEnsembleClassifier(n_estimators=100, random_state=1)

In [22]:
# Calculated the balanced accuracy score
y_pred_eec = eec_model.predict(X_test)
balanced_accuracy_score(y_test, y_pred_eec)

0.9270046452413471

In [23]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred_eec)

array([[   79,     8],
       [  925, 16193]], dtype=int64)

In [24]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred_eec))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.08      0.91      0.95      0.14      0.93      0.86        87
   low_risk       1.00      0.95      0.91      0.97      0.93      0.86     17118

avg / total       0.99      0.95      0.91      0.97      0.93      0.86     17205



### Final Questions

1. Which model had the best balanced accuracy score?

   The best balanced accuracy score belonged to the easy ensemble classifier.

2. Which model had the best recall score?
    The best recall score also belonged to the easy ensemble classifier.

3. Which model had the best geometric mean score?

    The best geometric mean score belonged to the balanced random forest classifier.

4. What are the top three features?

    The top 3 features are total received principal, last payment amount.